<a href="https://colab.research.google.com/github/Davioliveira1305/Simulacao-Estocastica/blob/main/Lista4_Simula%C3%A7%C3%A3o_Estoc%C3%A1stica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [31]:
%pip install simpy

# 1ª Questão

In [32]:
import numpy as np
import simpy
class Reparo:
  def __init__(self, env):
    self.env = env
    self.fila_formacao = simpy.Resource(env, capacity = 1) # Fila de formação eletrolítica
    self.fila_revestimento = simpy.Resource(env, capacity = 1) # Fila de revestimento
    self.fila_reparo = simpy.PriorityResource(env, capacity = 1) # Fila de reparo
    self.tam_fila_reparo = 0
    self.tam_fila_reparo_for = 0 # Tamanho da fila de reparos manuais(formação)
    self.tam_fila_reparo_reves = 0 # Tamanho da fila de reparos manuais(revestimento)
    self.tam_fila_formacao = 0
    self.tam_fila_revestimento = 0
    self.pedidos_no_sistema = 0
    self.tempo_confecc_inicial = [] # Marca o pedido e o tempo que ele começou o processo de confecção
    self.tempo_confecc_final = [] # Marca o pedido e o tempo que ele foi finalizado a sua confecção

  def carga_de_trabalho(self, tam_fila, pedidos_no_sistema):
    return (tam_fila * 100) / pedidos_no_sistema

  def produzir_formacao(self, pedido):
    self.tempo_confecc_inicial.append([pedido, self.env.now])
    self.pedidos_no_sistema = self.pedidos_no_sistema + 1
    self.tam_fila_formacao = self.tam_fila_formacao + 1
    print(f'Pedido {pedido} chegou na fila de formação em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}, Carga de Trabalho(formação) = {self.carga_de_trabalho(self.tam_fila_formacao, self.pedidos_no_sistema)} %')
    with self.fila_formacao.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(8, 12))
      self.tam_fila_formacao = self.tam_fila_formacao - 1
      if np.random.rand() < 0.15:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.reparo(pedido,'formacao'))
      else:
        print(f'Pedido {pedido} foi para o processo de revestimento em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.revestimento(pedido))

  def revestimento(self, pedido):
    self.tam_fila_revestimento = self.tam_fila_revestimento + 1
    print(f'Pedido {pedido} chegou na fila de revestimento em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}, Carga de Trabalho(revestimento) = {self.carga_de_trabalho(self.tam_fila_revestimento, self.pedidos_no_sistema)}%')
    with self.fila_revestimento.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(4, 8))
      self.tam_fila_revestimento = self.tam_fila_revestimento - 1
      if np.random.rand() < 0.12:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')
        self.env.process(self.reparo(pedido,'revestimento'))

      else:
        self.tempo_confecc_final.append([pedido, self.env.now])
        self.pedidos_no_sistema = self.pedidos_no_sistema - 1
        print(f'Pedido {pedido} ficou pronto em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')

  def reparo(self, pedido, tipo):
    self.tam_fila_reparo = self.tam_fila_reparo + 1
    print(f'Pedido {pedido} chegou no reparo em {self.env.now},  Carga de Trabalho(reparo) = {self.carga_de_trabalho(self.tam_fila_reparo, self.pedidos_no_sistema)}%')
    if tipo == 'formacao':
      prio = 0 # Processo de formação tem prioridade na fila de reparos
      self.tam_fila_reparo_for = self.tam_fila_reparo_for + 1
    elif tipo == 'revestimento':
      prio = 1
      self.tam_fila_reparo_reves = self.tam_fila_reparo_reves + 1
    with self.fila_reparo.request(priority = prio) as req:
      yield req
      if tipo == 'formacao':
        yield self.env.timeout(np.random.uniform(10, 20))
        print(f'Pedido {pedido} voltou para o processo de formação em {self.env.now}, Tamanho da fila de reparo(formação) = {self.tam_fila_reparo_for}')
        self.tam_fila_reparo_for = self.tam_fila_reparo_for - 1
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        self.env.process(self.produzir_formacao(pedido))
      else:
        yield self.env.timeout(np.random.uniform(15, 24))
        print(f'Pedido {pedido} voltou para o processo de revestimento em {self.env.now}, Tamanho da fila de reparo(revestimento) = {self.tam_fila_reparo_reves}')
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        self.tam_fila_reparo_reves = self.tam_fila_reparo_reves - 1
        self.env.process(self.revestimento(pedido))

def gera_pedidos(env, reparo):
  pedido = 0
  while True:
    yield env.timeout(np.random.triangular(3,7,14))
    pedido = pedido + 1
    env.process(reparo.produzir_formacao(pedido))

env = simpy.Environment()
fila_reparo = Reparo(env)

env.process(gera_pedidos(env, fila_reparo))

env.run(until=365*24)



Pedido 1 chegou na fila de formação em 5.537572173978173, Tamanho da fila de formação = 1, Carga de Trabalho(formação) = 100.0 %
Pedido 2 chegou na fila de formação em 13.645029680520008, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 100.0 %
Pedido 1 foi para o processo de revestimento em 16.104027330760207, Tamanho da fila de formação = 1
Pedido 1 chegou na fila de revestimento em 16.104027330760207, Tamanho da fila de revestimento = 1, Carga de Trabalho(revestimento) = 50.0%
Pedido 1 ficou pronto em 20.256991923973988, Tamanho da fila de revestimento = 0
Pedido 3 chegou na fila de formação em 22.995605020447254, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 100.0 %
Pedido 2 precisou de reparo em 26.10863418823299, Tamanho da fila de formação = 1
Pedido 2 chegou no reparo em 26.10863418823299,  Carga de Trabalho(reparo) = 50.0%
Pedido 4 chegou na fila de formação em 33.07270745894651, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 66.6

In [33]:
vetor_tempos = []
for i in fila_reparo.tempo_confecc_inicial:
  for j in fila_reparo.tempo_confecc_final:
    if i[0] == j[0]:
      vetor_tempos.append(j[1] - i[1])
media = sum(vetor_tempos)/len(vetor_tempos)
print(f'Em média um produto leva {media} horas para ser confeccionado.')

Em média um produto leva 1458.2968134746484 horas para ser confeccionado.


**Como o gargalo estava na fila de formação eletrolítica, adicionei mais uma máquina para essa função.**

In [34]:
import numpy as np
import simpy
class Reparo:
  def __init__(self, env):
    self.env = env
    self.fila_formacao = simpy.Resource(env, capacity = 2) # Fila de formação eletrolítica
    self.fila_revestimento = simpy.Resource(env, capacity = 1) # Fila de revestimento
    self.fila_reparo = simpy.PriorityResource(env, capacity = 1) # Fila de reparo
    self.tam_fila_reparo = 0
    self.tam_fila_reparo_for = 0 # Tamanho da fila de reparos manuais(formação)
    self.tam_fila_reparo_reves = 0 # Tamanho da fila de reparos manuais(revestimento)
    self.tam_fila_formacao = 0
    self.tam_fila_revestimento = 0
    self.pedidos_no_sistema = 0
    self.tempo_confecc_inicial = [] # Marca o pedido e o tempo que ele começou o processo de confecção
    self.tempo_confecc_final = [] # Marca o pedido e o tempo que ele foi finalizado a sua confecção

  def carga_de_trabalho(self, tam_fila, pedidos_no_sistema):
    return (tam_fila * 100) / pedidos_no_sistema

  def produzir_formacao(self, pedido):
    self.tempo_confecc_inicial.append([pedido, self.env.now])
    self.pedidos_no_sistema = self.pedidos_no_sistema + 1
    self.tam_fila_formacao = self.tam_fila_formacao + 1
    print(f'Pedido {pedido} chegou na fila de formação em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}, Carga de Trabalho(formação) = {self.carga_de_trabalho(self.tam_fila_formacao, self.pedidos_no_sistema)} %')
    with self.fila_formacao.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(8, 12))
      self.tam_fila_formacao = self.tam_fila_formacao - 1
      if np.random.rand() < 0.15:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.reparo(pedido,'formacao'))
      else:
        print(f'Pedido {pedido} foi para o processo de revestimento em {self.env.now}, Tamanho da fila de formação = {self.tam_fila_formacao}')
        self.env.process(self.revestimento(pedido))

  def revestimento(self, pedido):
    self.tam_fila_revestimento = self.tam_fila_revestimento + 1
    print(f'Pedido {pedido} chegou na fila de revestimento em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}, Carga de Trabalho(revestimento) = {self.carga_de_trabalho(self.tam_fila_revestimento, self.pedidos_no_sistema)}%')
    with self.fila_revestimento.request() as req:
      yield req
      yield self.env.timeout(np.random.uniform(4, 8))
      self.tam_fila_revestimento = self.tam_fila_revestimento - 1
      if np.random.rand() < 0.12:
        print(f'Pedido {pedido} precisou de reparo em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')
        self.env.process(self.reparo(pedido,'revestimento'))

      else:
        self.tempo_confecc_final.append([pedido, self.env.now])
        self.pedidos_no_sistema = self.pedidos_no_sistema - 1
        print(f'Pedido {pedido} ficou pronto em {self.env.now}, Tamanho da fila de revestimento = {self.tam_fila_revestimento}')

  def reparo(self, pedido, tipo):
    self.tam_fila_reparo = self.tam_fila_reparo + 1
    print(f'Pedido {pedido} chegou no reparo em {self.env.now},  Carga de Trabalho(reparo) = {self.carga_de_trabalho(self.tam_fila_reparo, self.pedidos_no_sistema)}%')
    if tipo == 'formacao':
      prio = 0 # Processo de formação tem prioridade na fila de reparos
      self.tam_fila_reparo_for = self.tam_fila_reparo_for + 1
    elif tipo == 'revestimento':
      prio = 1
      self.tam_fila_reparo_reves = self.tam_fila_reparo_reves + 1
    with self.fila_reparo.request(priority = prio) as req:
      yield req
      if tipo == 'formacao':
        yield self.env.timeout(np.random.uniform(10, 20))
        print(f'Pedido {pedido} voltou para o processo de formação em {self.env.now}, Tamanho da fila de reparo(formação) = {self.tam_fila_reparo_for}')
        self.tam_fila_reparo_for = self.tam_fila_reparo_for - 1
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        self.env.process(self.produzir_formacao(pedido))
      else:
        yield self.env.timeout(np.random.uniform(15, 24))
        print(f'Pedido {pedido} voltou para o processo de revestimento em {self.env.now}, Tamanho da fila de reparo(revestimento) = {self.tam_fila_reparo_reves}')
        self.tam_fila_reparo = self.tam_fila_reparo - 1
        self.tam_fila_reparo_reves = self.tam_fila_reparo_reves - 1
        self.env.process(self.revestimento(pedido))

def gera_pedidos(env, reparo):
  pedido = 0
  while True:
    yield env.timeout(np.random.triangular(3,7,14))
    pedido = pedido + 1
    env.process(reparo.produzir_formacao(pedido))

env = simpy.Environment()
fila_reparo = Reparo(env)

env.process(gera_pedidos(env, fila_reparo))

env.run(until=365*24)



A saída de streaming foi truncada nas últimas 5000 linhas.
Pedido 123 chegou na fila de formação em 949.0202096286532, Tamanho da fila de formação = 1, Carga de Trabalho(formação) = 5.0 %
Pedido 115 ficou pronto em 950.3220570172809, Tamanho da fila de revestimento = 2
Pedido 121 ficou pronto em 954.6958928430319, Tamanho da fila de revestimento = 1
Pedido 124 chegou na fila de formação em 956.2710877178079, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 10.526315789473685 %
Pedido 123 foi para o processo de revestimento em 959.0944542798845, Tamanho da fila de formação = 1
Pedido 123 chegou na fila de revestimento em 959.0944542798845, Tamanho da fila de revestimento = 2, Carga de Trabalho(revestimento) = 10.526315789473685%
Pedido 122 ficou pronto em 962.2992821787843, Tamanho da fila de revestimento = 1
Pedido 125 chegou na fila de formação em 962.9858154733536, Tamanho da fila de formação = 2, Carga de Trabalho(formação) = 10.526315789473685 %
Pedido 124 foi para o 

**Observe que com mais uma máquina para o estágio de formação eletrolítica, o tempo médio de confecção de um produto cai drasticamente.**

In [35]:
vetor_tempos = []
for i in fila_reparo.tempo_confecc_inicial:
  for j in fila_reparo.tempo_confecc_final:
    if i[0] == j[0]:
      vetor_tempos.append(j[1] - i[1])
media = sum(vetor_tempos)/len(vetor_tempos)
print(f'Em média um produto leva {media} horas para ser confeccionado.')

Em média um produto leva 30.98237111533791 horas para ser confeccionado.
